# 🍎 Fruit Classification — Multi-Class CNN using TensorFlow & Keras

This Colab-ready notebook trains a simple CNN to classify **Apple**, **Banana**, and **Orange** images. It includes data loading (Drive), training with callbacks, evaluation, single-image prediction, and TFLite export.

> **Note:** Run this in Google Colab. Mount your Drive and place your dataset in `MyDrive/fruits-dataset/` with `train/` and `validation/` subfolders, each containing `apple/`, `banana/`, and `orange/` folders.


## 1) Setup & Imports

In [ ]:
# Mount Google Drive (Colab)
from google.colab import drive, files
drive.mount('/content/drive')

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import numpy as np, os, itertools

print('# ➤ TensorFlow version:', tf.__version__)

## 2) Data Loading & Preprocessing

Set paths to your dataset in Drive. The directory structure should be:
```
fruits-dataset/
  train/
    apple/
    banana/
    orange/
  validation/
    apple/
    banana/
    orange/
```


In [ ]:
# Paths (edit if your location differs)
train_dir = '/content/drive/MyDrive/fruits-dataset/train'
val_dir   = '/content/drive/MyDrive/fruits-dataset/validation'

# Parameters
IMG_SIZE = (100, 100)
BATCH_SIZE = 32

# Generators
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True)

val_datagen = ImageDataGenerator(rescale=1./255)

train_gen = train_datagen.flow_from_directory(train_dir,
                                              target_size=IMG_SIZE,
                                              batch_size=BATCH_SIZE,
                                              class_mode='categorical',
                                              shuffle=True)

val_gen = val_datagen.flow_from_directory(val_dir,
                                          target_size=IMG_SIZE,
                                          batch_size=BATCH_SIZE,
                                          class_mode='categorical',
                                          shuffle=False)

print('\n# ➤ Classes found (label -> index):', train_gen.class_indices)

## 3) Build the CNN Model

In [ ]:
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(IMG_SIZE[0], IMG_SIZE[1], 3)),
    MaxPooling2D(2,2),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(len(train_gen.class_indices), activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

## 4) Callbacks

In [ ]:
callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True, verbose=1),
    ModelCheckpoint('/content/drive/MyDrive/fruit_best_model.h5', monitor='val_accuracy', save_best_only=True, verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1)
]

## 5) Train the Model

In [ ]:
EPOCHS = 25

history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=EPOCHS,
    callbacks=callbacks,
    verbose=1
)

## 6) Plot Training History

In [ ]:
plt.figure(figsize=(12,5))

plt.subplot(1,2,1)
plt.plot(history.history['accuracy'], label='train_acc')
plt.plot(history.history['val_accuracy'], label='val_acc')
plt.title('Accuracy')
plt.legend()

plt.subplot(1,2,2)
plt.plot(history.history['loss'], label='train_loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.title('Loss')
plt.legend()

plt.show()

## 7) Evaluation — Confusion Matrix & Classification Report

In [ ]:
# Predict on validation set
val_gen.reset()
preds = model.predict(val_gen, verbose=1)
y_pred = np.argmax(preds, axis=1)
y_true = val_gen.classes

labels = list(val_gen.class_indices.keys())

# Classification report
print('\nClassification Report:\n')
print(classification_report(y_true, y_pred, target_names=labels))

# Confusion matrix heatmap
cm = confusion_matrix(y_true, y_pred)

plt.figure(figsize=(6,5))
plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Confusion matrix')
plt.colorbar()
tick_marks = np.arange(len(labels))
plt.xticks(tick_marks, labels, rotation=45)
plt.yticks(tick_marks, labels)

thresh = cm.max() / 2.
for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
    plt.text(j, i, format(cm[i, j], 'd'),
             horizontalalignment='center',
             color='white' if cm[i, j] > thresh else 'black')

plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.tight_layout()
plt.show()

## 8) Single Image Prediction (Upload an image in Colab)

In [ ]:
from tensorflow.keras.preprocessing import image

uploaded = files.upload()
img_path = list(uploaded.keys())[0]

img = image.load_img(img_path, target_size=IMG_SIZE)
plt.imshow(img); plt.axis('off'); plt.show()

x = image.img_to_array(img) / 255.0
x = np.expand_dims(x, axis=0)

pred = model.predict(x)
label_map = {v:k for k,v in train_gen.class_indices.items()}
pred_label = label_map[np.argmax(pred)]
print(f'Predicted: {pred_label} | Confidence: {np.max(pred)*100:.2f}%')

## 9) Export Model to TensorFlow Lite

In [ ]:
# Save Keras model (best already saved by checkpoint)
model.save('/content/drive/MyDrive/fruit_model_final.h5')

# Convert to TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

open('fruit_model_android.tflite', 'wb').write(tflite_model)

# Download the TFLite file
files.download('fruit_model_android.tflite')

## 10) Summary

- Trained a 3-class CNN (Apple, Banana, Orange).
- Added callbacks and evaluation metrics.
- Exported model to `.tflite` for Android deployment.

Good luck — run this in Colab and let me know if you want optimizations or smaller model variants for mobile.